# Head Pose Estimation- Project

- In this project we used AFLW2000 Dataset to read image and Extraxt landmarks from images
- AFLW2000 conist of 2000 images and you can download from this link 
http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/main.htm

- In this NoteBook steps we follow:
  - Import libraries
  - Read Images and Extract Landmarks
  - Read and Prepare Data
  - Train Machine Learning Models
  - Save models 
  - Live Camera and Draw axises on faces


### Team Members:
   - Ahmad Khaled
   - Aya Hassan

In [13]:
# Install mediaPiPe and Open Cv

# !pip install mediapipe opencv-python

# Import libararies

In [14]:
import mediapipe as mp
import cv2
import glob
import numpy as np
import csv
import os
from mat4py import loadmat
import scipy.io as sio
import math

### You can read more about holistic media pipe model from this Link
- https://google.github.io/mediapipe/solutions/holistic.html

In [15]:
# Drawing helpers in mediapipe
mp_drawing = mp.solutions.drawing_utils

# Mediapipe Solutions
mp_holistic = mp.solutions.holistic 

mp_drawing_styles = mp.solutions.drawing_styles


In [16]:
paths = []
# get the path/directory
folder_dir ="C:\\Users\\ahmad\Downloads\AFLW2000"

# iterate over files in
# that directory
for images in glob.iglob(f'{folder_dir}/*'):

    # check if the image ends with png
    if (images.endswith(".jpg")):
#         print(images)
        paths.append(images)

In [17]:
# Check Number of Paths in dataset directory
len(paths)

2000

# Try media Pipe Model:
- we try mediapipe model on our dataset ALFW2000 to learn how to access it and get landmarks from it
- we try it on first image, so we use break statement

In [18]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 

    for idx, file in enumerate(paths):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        # Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#         results = holistic.process(image)
#         print(results.face_landmarks)
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        
        break


In [19]:
# access Landmarks to use it 
results.face_landmarks.landmark[0]

x: 0.47754186391830444
y: 0.6920552253723145
z: -0.018991734832525253

In [20]:
# Print Number of Landmarks extracted From Images which 468 points
num_coords = len(results.face_landmarks.landmark)
num_coords

468

In [21]:
landmarks = []

# Create a list consist of (x1,y1) -> (x468,y468)
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val)]
# landmarks

# insert lables in this list
angles=['pitch','yaw','roll']
data = landmarks+angles

In [22]:
# Create CSV File with headers using the list of Landmarks as our CSV file headers

with open('landmarks_data.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(data)

In [23]:
# split to get the name of the file 
paths[0].split('\\')[-1].replace('.jpg','')

'image00002'

## Read Images & Extract Landmarks

In [24]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for idx, file in enumerate(paths):
        count=0
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
#         Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        try:
           
            # Get all face landmarks points axis(xn , yn) n =1 -> 468 
            face = results.face_landmarks.landmark
            
            # get x-axis value of Nose
            NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width
            
            # get y-axis value of Nose
            NoseY=results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y   *image_height
            
            # get x-axis value of left eye outer
            LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x *image_width
            
            # get y-axis value of left eye outer
            LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y* image_height
            
            # Calculate the distance between Nose and Outer Left Eye
            dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])
        
            
            
            
            
            # In this block Normalize Data getting from Image in Dataset anf flatten it to 1-D array
            face_row = list(np.array([[   ((landmark.x *image_width)-NoseX)/dist, 
                                            ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).flatten())
            
            # Get the name of file for ex:"image2000"
            random_file = file.split('\\')[-1].replace('.jpg' , '')
            
            # Read the Matlab file of the image to get labels
            mat_file = sio.loadmat('C:\\Users\\ahmad\\Downloads\\AFLW2000\\'+ random_file+ '.mat')
            
            # We can find labels(Pitch, Yaw , Roll) angles in pose_para in the mat file
            pose_para = mat_file["Pose_Para"][0][:3]
            
            # get pitch angle
            pitch = pose_para[0]
            
            # get Yaw angle
            yaw = pose_para[1]
            
            # Get Roll angle
            roll = pose_para[2]
            
            # Append lables in the list named face_row
            face_row.insert(((468*2)+0),pitch)
            face_row.insert(((468*2)+1),yaw)
            face_row.insert(((468*2)+2),roll)
            
            # append the Face_row list to our CSV file
            with open('landmarks_data.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(face_row) 
                
        except:
            pass
        

# Read and Prepare Data

In [26]:
# Import pandas to deal with data 
import pandas as pd

# import train_test_split to split our data to train,validation,test datasets
from sklearn.model_selection import train_test_split

In [27]:
#Read CSV file as DataFrame
df=pd.read_csv('landmarks_data.csv')
df.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y465,x466,y466,x467,y467,x468,y468,pitch,yaw,roll
0,-0.159295,0.300003,-0.169120,0.067671,-0.146352,0.109180,-0.190952,-0.301066,-0.161892,-0.022124,...,-0.640003,0.132219,-0.604989,0.751723,-0.761274,0.807323,-0.806983,-0.399231,0.018227,0.085676
1,-0.334478,0.261430,-0.472663,-0.051103,-0.284650,0.053159,-0.358159,-0.332285,-0.480441,-0.137888,...,-0.525273,0.037087,-0.505745,0.562256,-0.565438,0.632099,-0.617696,0.470065,1.189533,0.300959
2,-0.981481,1.076963,-0.956155,0.954189,-0.976846,0.985318,-1.013231,0.778098,-0.954629,0.903266,...,0.634062,-0.919076,0.647000,-0.675460,0.588512,-0.651586,0.567591,-0.184650,0.881137,-0.236852
3,0.127447,0.433119,-0.086203,0.200610,0.035008,0.228611,-0.239562,-0.172509,-0.133581,0.104239,...,-0.642603,0.021166,-0.595805,0.629668,-0.972822,0.673793,-1.040024,-0.175379,0.299208,-0.373374
4,-0.078761,0.400796,-0.344684,0.090758,-0.104594,0.147864,-0.317465,-0.235742,-0.383646,-0.000681,...,-0.595326,-0.004369,-0.552671,0.427108,-0.790101,0.482372,-0.853066,-0.882169,1.198003,-1.033374


In [28]:
#Features
x=df.drop(['pitch','yaw','roll'], axis=1)
#Pitch label
yp=df['pitch']
#Yaw label
yy=df['yaw']
#roll label
yr=df['roll']
#labels
yall =df[['pitch','yaw','roll']]

In [29]:
#************************************************ PITCH ****************************************************************

X_train_p, X_val_p, y_train_p, y_val_p = train_test_split(x, yp, test_size=0.2,shuffle=True, random_state=1234)

X_val_p, X_test_p, y_val_p, y_test_p = train_test_split(X_val_p, y_val_p, test_size=0.5,shuffle=True, random_state=1234)


#************************************************* YAW ******************************************************************

X_train_y, X_val_y, y_train_y, y_val_y = train_test_split(x, yy, test_size=0.2, shuffle=True,random_state=1234)

X_val_y, X_test_y, y_val_y, y_test_y = train_test_split(X_val_y, y_val_y, test_size=0.5, shuffle=True,random_state=1234)


#*********************************************** Roll *******************************************************************

X_train_r, X_val_r, y_train_r, y_val_r = train_test_split(x, yr, test_size=0.2,shuffle=True, random_state=1234)

X_val_r, X_test_r, y_val_r, y_test_r = train_test_split(X_val_r, y_val_r, test_size=0.5,shuffle=True, random_state=1234)


#*************************************************** ALL ****************************************************************


X_train_all, X_val_all, y_train_all, y_val_all = train_test_split(x, yall, test_size=0.2,shuffle=True, random_state=1234)

X_val_all, X_test_all, y_val_all, y_test_all = train_test_split(X_val_all, y_val_all, test_size=0.5,shuffle=True, random_state=1234)



# Train Machine Learning Models

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer, PowerTransformer, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge , ridge_regression,Lasso
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor,BaggingRegressor,ExtraTreesRegressor
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
import sklearn.linear_model
from sklearn.impute import KNNImputer

In [33]:
# Create Models to Use in PiPeLine
rf = RandomForestRegressor()

bagging = BaggingRegressor(base_estimator=ExtraTreesRegressor(),n_estimators=10, random_state=0)

boosting = GradientBoostingRegressor()

svm= SVR()

adaboost = AdaBoostRegressor()

trees = ExtraTreesRegressor()

lr = LinearRegression()

# YAW

In [35]:
#Create PipeLine to train our Data for Yaw angle
pipelines = {
        'rf_pca':make_pipeline(StandardScaler(),PCA(n_components=0.98), 
        VotingRegressor(estimators=[('RF',rf),('Bagging',bagging),('Boosting',boosting)]))
            
}



In [36]:
#create a dictionary of trained models and its name
yaw_fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_y, y_train_y)
    yaw_fit_models[algo] = model


In [37]:
#use model on our train Dataset
Train_pred_yaw = yaw_fit_models['rf_pca'].predict(X_train_y)

#use model on our Validation Dataset
Validation_pred_yaw = yaw_fit_models['rf_pca'].predict(X_val_y)

#use model on our test Dataset
yaw_hat=yaw_fit_models['rf_pca'].predict(X_test_y)

In [38]:
print("Train MSE "       , mean_squared_error( y_train_y,Train_pred_yaw ))
print("Validation MSE "  , mean_squared_error( y_val_y,Validation_pred_yaw ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_y,Train_pred_yaw))
print("Validation r2_score "       , r2_score( y_val_y, Validation_pred_yaw))
print("Test r2_score "             , r2_score(y_test_y , yaw_hat))
print("\n###########################\n")

print("Train score "                   , yaw_fit_models['rf_pca'].score(X_train_y , y_train_y))
print("Validation Score "              , yaw_fit_models['rf_pca'].score(X_val_y, y_val_y))
print("test score: "                   ,yaw_fit_models ['rf_pca'].score(X_test_y, y_test_y))
 
print("\n###########################\n")

Train MSE  0.010177408538945259
Validation MSE  0.01817336333745248

###########################

Train r2_score  0.9695246337389682
Validation r2_score  0.947810332177787
Test r2_score  0.9459211910094788

###########################

Train score  0.9695246337389682
Validation Score  0.947810332177787
test score:  0.9459211910094788

###########################



In [44]:
import pickle 

#Saving model in pickle file, so we can use it just read it and use it
with open('yaw_model.pkl', 'wb') as f:
    pickle.dump(yaw_fit_models ['rf_pca'], f)



# Roll

In [45]:
pipelines = {

    # 'rfpcap' :make_pipeline(PCA(n_components=0.99), RandomForestRegressor(n_estimators=200)),
    'rfpcap':make_pipeline(MinMaxScaler(),PCA(n_components=0.98), 
    VotingRegressor(estimators=[('RF',rf),('Bagging',bagging),('Boosting',boosting),('SVM',svm)]))

}


In [46]:
roll_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_r, y_train_r)
    roll_fit_models[algo] = model


In [47]:
Train_pred_roll     = roll_fit_models  ['rfpcap'].predict(X_train_r)
Validation_pred_roll = roll_fit_models ['rfpcap'].predict(X_val_r)
roll_hat             =roll_fit_models  ['rfpcap'].predict(X_test_r)

In [48]:
print("Train MSE "       , mean_squared_error( y_train_r,Train_pred_roll ))
print("Validation MSE "  , mean_squared_error( y_val_r,Validation_pred_roll ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_r,Train_pred_roll))
print("Validation r2_score "       , r2_score( y_val_r, Validation_pred_roll))
print("Test r2_score "             , r2_score(y_test_r , roll_hat))
print("\n###########################\n")

print("Train score "                   , roll_fit_models['rfpcap'].score(X_train_r , y_train_r))
print("Validation Score "              , roll_fit_models['rfpcap'].score(X_val_r, y_val_r))
print("test score: "                   , roll_fit_models ['rfpcap'].score(X_test_r, y_test_r))
 
print("\n###########################\n")


Train MSE  0.09518376248786642
Validation MSE  0.013720471616231173

###########################

Train r2_score  0.8112878506244512
Validation r2_score  0.8185937091383267
Test r2_score  0.7323251235314374

###########################

Train score  0.8112878506244512
Validation Score  0.8185937091383267
test score:  0.7323251235314374

###########################



In [49]:
with open('roll_model.pkl', 'wb') as f:
    pickle.dump(roll_fit_models ['rfpcap'], f)

# PITCH

In [50]:
from sklearn import tree
# ,('Boosting',boosting)
# ,('Trees',trees)
pipelines = {
    'rf_pca':make_pipeline(MinMaxScaler(),PCA(n_components=0.99), RandomForestRegressor()),
    # 'rf_pca':make_pipeline(MinMaxScaler(),PCA(n_components=0.99), VotingRegressor(estimators=[('Bagging',bagging),('SVM',svm),('Boosting',boosting),('Trees',trees)]))

}



In [51]:
pitch_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_p, y_train_p)
    pitch_fit_models[algo] = model


In [52]:
Train_pred_pitch = pitch_fit_models['rf_pca'].predict(X_train_p)
Validation_pred_pitch = pitch_fit_models['rf_pca'].predict(X_val_p)
pitch_hat=pitch_fit_models['rf_pca'].predict(X_test_p)

In [53]:


print("Train MSE "       , mean_squared_error( y_train_p, Train_pred_pitch))
print("Validation MSE "  , mean_squared_error( y_val_p,Validation_pred_pitch ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_p,Train_pred_pitch))
print("Validation r2_score "       , r2_score( y_val_p, Validation_pred_pitch))
print("Test r2_score "             , r2_score(y_test_p , pitch_hat))
print("\n###########################\n")

print("Train score "                   , pitch_fit_models['rf_pca'].score(X_train_p , y_train_p))
print("Validation Score "              , pitch_fit_models['rf_pca'].score(X_val_p, y_val_p))
print("test score: "                   , pitch_fit_models ['rf_pca'].score(X_test_p, y_test_p))
 
print("\n###########################\n")

Train MSE  0.051302664824360036
Validation MSE  0.02055480386181735

###########################

Train r2_score  0.857991010129622
Validation r2_score  0.6792719275738812
Test r2_score  0.6299871503728327

###########################

Train score  0.857991010129622
Validation Score  0.6792719275738812
test score:  0.6299871503728327

###########################



In [54]:
with open('pitch_model.pkl', 'wb') as f:
    pickle.dump(pitch_fit_models ['rf_pca'], f)

In [55]:
from math import cos ,sin

# Read Trained  MODELS

In [56]:
with open('yaw_model.pkl', 'rb') as f:
    yaw_model = pickle.load(f)

In [57]:
with open('pitch_model.pkl', 'rb') as f:
    pitch_model = pickle.load(f)

In [58]:
with open('roll_model.pkl', 'rb') as f:
    roll_model = pickle.load(f)

# Live Cam and Draw axis 

In [59]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_height, image_width, _ = image.shape

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        
        
        
        
        try:
           
            face = results.face_landmarks.landmark
            NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width
            NoseY=results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y   *image_height
            LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x *image_width
            LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y* image_height
            
            dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])
            face_row = list(np.array([[   ((landmark.x *image_width)-NoseX)/dist, 
                                            ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).reshape(1, -1))
            
#             print(face_row)
            pitch = pitch_model.predict(face_row)
            yaw = yaw_model.predict(face_row)
            roll = roll_model.predict(face_row)
            

#             cv2_imshow(draw_axis(image,pitch,yaw,roll))
            yaw = -yaw
            tdx = NoseX
            tdy = NoseY
            size=100
    # X-Axis pointing to right. drawn in red
            x1 = size * (cos(yaw) * cos(roll)) + tdx
            y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
            x2 = size * (-cos(yaw) * sin(roll)) + tdx
            y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
            x3 = size * (sin(yaw)) + tdx
            y3 = size * (-cos(yaw) * sin(pitch)) + tdy
            cv2.line(image, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
            cv2.line(image, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
            cv2.line(image, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

            
            

                
        except:
            pass


        
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()